In [1]:

from pynq import Overlay, allocate
import numpy as np
import time

class FisherfaceDriver:
    def __init__(self, bitfile_path):
        self.overlay = Overlay(bitfile_path)
        self.dma = self.overlay.axi_dma_0
        self.ip = self.overlay.fisherface_accel_0
        
        self.VECTOR_SIZE = 10000
        
        # --- FIX: PREVENT OVERFLOW ---
        self.HW_SCALE = 128.0
        # Reduced from 100.0 to 20.0 to ensure results fit in 16-bit accumulator
        # This fixes the Test 2 drift AND the Test 3 crash.
        self.SW_BOOST = 20.0 
        self.TOTAL_SCALE = self.HW_SCALE * self.SW_BOOST
        
        self.input_buffer = allocate(shape=(self.VECTOR_SIZE,), dtype=np.int32)
        
    def _send_chunked(self, data_array, is_weight=False):
        flat_data = data_array.flatten()
        
        # Apply Boost
        scale = self.TOTAL_SCALE if is_weight else self.HW_SCALE
        
        # Quantize
        quantized = (flat_data * scale).astype(np.int32)
        np.copyto(self.input_buffer, quantized)
        
        # Send Chunks
        CHUNK_SIZE = 4000
        total_elements = self.VECTOR_SIZE
        for i in range(0, total_elements, CHUNK_SIZE):
            end = min(i + CHUNK_SIZE, total_elements)
            self.dma.sendchannel.transfer(self.input_buffer[i:end])
            self.dma.sendchannel.wait()

    def load_mean(self, mean_vector):
        self.ip.register_map.mode.mode = 1
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(mean_vector, is_weight=False)
        
    def load_weights(self, weight_vector):
        self.ip.register_map.mode.mode = 2
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(weight_vector, is_weight=True)
        
    def inference(self, face_vector):
        self.ip.register_map.mode.mode = 0
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(face_vector, is_weight=False)
        
        while self.ip.register_map.CTRL.AP_DONE == 0: pass
            
        raw_int = int(self.ip.register_map.output_score.output_score)
        if raw_int >= 0x80000000: raw_int -= 0x100000000
            
        # De-scale using the new boost value
        # Math: Result_Int / (2^16 * 20.0)
        acc_scale = 65536.0 # 2^16
        result_float = raw_int / (acc_scale * self.SW_BOOST)
        return result_float

In [2]:
import numpy as np

# 1. Load Data
print("Loading Model Data...")
data = np.load("face_model_data2.npz", allow_pickle=True)

mean_vec = data['mean_vec']
eigen_vecs = data['eigen_vecs']
train_proj_db = data['train_proj_db']
train_lbls_db = data['train_lbls_db']
label_names = data['label_names'].item()

# 2. FIX THE SHAPE (SLICING)
# The diagnosis showed eigen_vecs is (10000, 2).
# We take only the FIRST column [:, 0] to match the 1D hardware.
if eigen_vecs.ndim > 1:
    print(f"Slicing Model from {eigen_vecs.shape} to 1D...")
    w_vector = eigen_vecs[:, 0]        # Take 1st feature for HW
    
    # CRITICAL: We must also slice the database to match!
    # train_proj_db is likely (N, 2). We take (N, 1) and flatten to (N,)
    train_proj_db = train_proj_db[:, 0]
else:
    w_vector = eigen_vecs

# 3. Initialize & Load
driver = FisherfaceDriver("fisherface.bit")
driver.load_mean(mean_vec)
driver.load_weights(w_vector)

print("HARDWARE READY: Parameters loaded with Precision Boost.")

Loading Model Data...
Slicing Model from (10000, 2) to 1D...


HARDWARE READY: Parameters loaded with Precision Boost.


In [3]:
import numpy as np

# 1. Load the Training Data
print("Loading Model Data from NPZ...")
data = np.load("face_model_data2.npz", allow_pickle=True)

mean_vec = data['mean_vec']          
eigen_vecs = data['eigen_vecs']      
train_proj_db = data['train_proj_db'] 
train_lbls_db = data['train_lbls_db'] 
label_names = data['label_names'].item() 

print("Model Loaded. Initializing Hardware...")

# 2. Initialize Driver
# MAKE SURE 'fisherface.bit' IS IN THE SAME FOLDER
driver = FisherfaceDriver("fisherface.bit")

# 3. Load Parameters to FPGA
# Handle Shape: If eigen_vecs is (10000, 1) or (10000,), it's fine.
# If it is (10000, K), we take the first column because HW is 1D.
if eigen_vecs.ndim > 1:
    w_vector = eigen_vecs[:, 0]
else:
    w_vector = eigen_vecs

driver.load_mean(mean_vec)
driver.load_weights(w_vector)

print("HARDWARE READY: Parameters loaded to FPGA.")

Loading Model Data from NPZ...
Model Loaded. Initializing Hardware...
HARDWARE READY: Parameters loaded to FPGA.


In [5]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import threading
import time
from collections import Counter

# --------------------------------------------------------------
# CONFIGURATION
# --------------------------------------------------------------
IMG_SIZE = (100, 100)
THRESHOLD = 300.0    
SCALE_FACTOR = 0.5   
BATCH_SIZE = 10      # <--- NEW: Process 10 frames before changing the name label

# --------------------------------------------------------------
# SETUP 1: LOAD HAAR CASCADE
# --------------------------------------------------------------
cascade_path = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cascade_path)

if face_cascade.empty():
    print("CRITICAL ERROR: Could not find 'haarcascade_frontalface_default.xml'.")
    class DummyCascade:
        def detectMultiScale(self, *args): return []
    face_cascade = DummyCascade()
else:
    print("Face Cascade loaded successfully.")

# --------------------------------------------------------------
# SHARED GLOBAL VARIABLES
# --------------------------------------------------------------
current_frame = None           
latest_result_box = None       
latest_result_name = ""        
latest_result_color = (255,255,255)
system_running = True          

frame_lock = threading.Lock() 

# --------------------------------------------------------------
# HELPER: Preprocessing
# --------------------------------------------------------------
def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim == 3 else img
    g = cv2.resize(g, IMG_SIZE, interpolation=cv2.INTER_AREA)
    g = cv2.equalizeHist(g)
    g = cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX)
    return g

# --------------------------------------------------------------
# THREAD 1: THE "BRAIN" (Runs in Background)
# --------------------------------------------------------------
def ai_processing_worker():
    global latest_result_box, latest_result_name, latest_result_color, current_frame
    
    # Local buffer to store the last 10 votes
    prediction_buffer = [] 
    
    print("[THREAD] AI Worker Started")
    
    while system_running:
        frame_to_process = None
        with frame_lock:
            if current_frame is not None:
                frame_to_process = current_frame.copy()
        
        if frame_to_process is None:
            time.sleep(0.05) # Wait if camera hasn't started
            continue
            
        # Resize for Speed
        small_frame = cv2.resize(frame_to_process, (0,0), fx=SCALE_FACTOR, fy=SCALE_FACTOR)
        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        
        # Detection
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        
        if len(faces) > 0:
            largest_face = max(faces, key=lambda r: r[2]*r[3])
            (sx, sy, sw, sh) = largest_face
            
            # Scale up coordinates
            scale = 1 / SCALE_FACTOR
            x, y, w, h = int(sx*scale), int(sy*scale), int(sw*scale), int(sh*scale)
            
            # Bounds Check
            if y+h < frame_to_process.shape[0] and x+w < frame_to_process.shape[1]:
                # HARDWARE CALL
                face_roi = cv2.cvtColor(frame_to_process[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
                processed_face = preprocess(face_roi)
                flat_face = processed_face.reshape(-1)
                
                # FPGA INFERENCE
                test_proj_scalar = driver.inference(flat_face)
                
                # CLASSIFICATION (Instant Calculation)
                dists = np.linalg.norm(train_proj_db - test_proj_scalar, axis=1)
                min_index = np.argmin(dists)
                min_dist = dists[min_index]
                predicted_lbl = train_lbls_db[min_index]
                
                # Determine "Instant" Result
                if min_dist > THRESHOLD:
                    instant_name = "Unknown"
                else:
                    instant_name = label_names[predicted_lbl]
                    if instant_name.lower() == "unknown": instant_name = "Unknown"

                # --- BATCH VOTING LOGIC START ---
                prediction_buffer.append(instant_name)

                # Only change the displayed name when we have 10 votes
                if len(prediction_buffer) >= BATCH_SIZE:
                    counts = Counter(prediction_buffer)
                    winner_name, vote_count = counts.most_common(1)[0]
                    
                    # Logic: If Unknown wins OR the winner has weak support (<6 votes), reject it
                    if winner_name == "Unknown" or vote_count < 6:
                        latest_result_name = "Unknown"
                        latest_result_color = (0, 0, 255) # Red
                    else:
                        latest_result_name = f"{winner_name} ({min_dist:.0f})"
                        latest_result_color = (0, 255, 0) # Green
                    
                    # Reset buffer for next batch
                    prediction_buffer = []
                # --- BATCH VOTING LOGIC END ---
                
                # ALWAYS update the box immediately so it follows the face smoothly
                latest_result_box = (x, y, w, h)
                
        else:
            # If face is lost, clear everything immediately
            prediction_buffer = []
            latest_result_box = None
            # We don't clear the name immediately so you can read the last result
            # but you can uncomment the next line if you want it to disappear instantly
            # latest_result_name = ""

        time.sleep(0.01) # Yield CPU

# --------------------------------------------------------------
# THREAD 2: THE "EYES" (Main Jupyter Loop)
# --------------------------------------------------------------
image_widget = widgets.Image(format='jpeg', width=640, height=480)
display(image_widget)

cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    system_running = True
    worker_thread = threading.Thread(target=ai_processing_worker)
    worker_thread.start()

    print("[INFO] System Running! Press 'Stop' in Jupyter to end.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret: break
            
            with frame_lock:
                current_frame = frame
            
            # Draw Overlay
            if latest_result_box is not None:
                (x, y, w, h) = latest_result_box
                cv2.rectangle(frame, (x, y), (x+w, y+h), latest_result_color, 2)
                cv2.putText(frame, latest_result_name, (x, y-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, latest_result_color, 2)
                
            _, encoded_image = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
            image_widget.value = encoded_image.tobytes()
            
            time.sleep(0.001) 

    except KeyboardInterrupt:
        print("Stopping...")

    finally:
        system_running = False
        worker_thread.join()
        cap.release()
        print("[INFO] System Stopped")

Face Cascade loaded successfully.


Image(value=b'', format='jpeg', height='480', width='640')

Error: Could not open camera.


[ WARN:0] global ./modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [5]:
import numpy as np
from pynq import Overlay, allocate
import time

# ==============================================================================
# 1. HARDWARE DRIVER (The Exact Version from your Untitled.ipynb)
# ==============================================================================
class FisherfaceDriver:
    def __init__(self, bitfile_path):
        self.overlay = Overlay(bitfile_path)
        self.dma = self.overlay.axi_dma_0
        self.ip = self.overlay.fisherface_accel_0
        
        self.VECTOR_SIZE = 10000
        # Precision Boost Configuration
        self.HW_SCALE = 128.0
        self.SW_BOOST = 20.0 
        self.TOTAL_SCALE = self.HW_SCALE * self.SW_BOOST
        
        self.input_buffer = allocate(shape=(self.VECTOR_SIZE,), dtype=np.int32)
        
    def _send_chunked(self, data_array, is_weight=False):
        flat_data = data_array.flatten()
        scale = self.TOTAL_SCALE if is_weight else self.HW_SCALE
        
        # Quantize
        quantized = (flat_data * scale).astype(np.int32)
        np.copyto(self.input_buffer, quantized)
        
        # Send in Chunks
        CHUNK_SIZE = 4000
        total_elements = self.VECTOR_SIZE
        for i in range(0, total_elements, CHUNK_SIZE):
            end = min(i + CHUNK_SIZE, total_elements)
            self.dma.sendchannel.transfer(self.input_buffer[i:end])
            self.dma.sendchannel.wait()

    def load_mean(self, mean_vector):
        self.ip.register_map.mode.mode = 1
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(mean_vector, is_weight=False)
        
    def load_weights(self, weight_vector):
        self.ip.register_map.mode.mode = 2
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(weight_vector, is_weight=True)
        
    def inference(self, face_vector):
        self.ip.register_map.mode.mode = 0
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(face_vector, is_weight=False)
        
        while self.ip.register_map.CTRL.AP_DONE == 0: pass
            
        raw_int = int(self.ip.register_map.output_score.output_score)
        if raw_int >= 0x80000000: raw_int -= 0x100000000
        
        # De-scale
        acc_scale = 65536.0 # 2^16
        result_float = raw_int / (acc_scale * self.SW_BOOST)
        return result_float

# ==============================================================================
# 2. SOFTWARE IMPLEMENTATION (Golden Model)
# ==============================================================================
def software_inference(face, mean, weights):
    # Standard Fisherface Projection: z = (x - mean) . W
    centered = face - mean
    result = np.dot(centered, weights)
    return result

# ==============================================================================
# 3. TEST HARNESS
# ==============================================================================

print("--- STARTING VERIFICATION ---")

# A. LOAD REAL DATA
print("1. Loading NPZ Data...")
data = np.load("face_model_data2.npz", allow_pickle=True)
mean_vec = data['mean_vec']
eigen_vecs = data['eigen_vecs']

# Handle 2D Weights (Slice 1st column)
if eigen_vecs.ndim > 1:
    print(f"   Note: Slicing eigen_vecs from {eigen_vecs.shape} to 1D.")
    w_vec = eigen_vecs[:, 0]
else:
    w_vec = eigen_vecs

# B. INITIALIZE HARDWARE
print("2. Initializing Hardware...")
driver = FisherfaceDriver("fisherface.bit")
driver.load_mean(mean_vec)
driver.load_weights(w_vec)
print("   Hardware Loaded.")

# C. RUN TESTS
print("\n3. Running Comparison Tests...")

# Test Case 1: The Mean Face (Expected Result approx 0.0)
print("\n[TEST 1] Input = Mean Face")
input_1 = mean_vec.copy()
sw_res_1 = software_inference(input_1, mean_vec, w_vec)
hw_res_1 = driver.inference(input_1)
print(f"   SW Result: {sw_res_1:.4f}")
print(f"   HW Result: {hw_res_1:.4f}")
print(f"   Diff:      {abs(sw_res_1 - hw_res_1):.4f}")

# Test Case 2: Random Face (Stress Test)
print("\n[TEST 2] Input = Random Noise (0-255)")
np.random.seed(42) # Fixed seed for reproducibility
input_2 = np.random.rand(10000) * 255.0
sw_res_2 = software_inference(input_2, mean_vec, w_vec)
hw_res_2 = driver.inference(input_2)
print(f"   SW Result: {sw_res_2:.4f}")
print(f"   HW Result: {hw_res_2:.4f}")
print(f"   Diff:      {abs(sw_res_2 - hw_res_2):.4f}")

# Test Case 3: Synthetic Face (Mean + Weights)
# Result should be roughly sum(weights^2)
print("\n[TEST 3] Input = Mean + Weights * 1000")
input_3 = mean_vec + (w_vec * 1000.0)
sw_res_3 = software_inference(input_3, mean_vec, w_vec)
hw_res_3 = driver.inference(input_3)
print(f"   SW Result: {sw_res_3:.4f}")
print(f"   HW Result: {hw_res_3:.4f}")
print(f"   Diff:      {abs(sw_res_3 - hw_res_3):.4f}")

# ==============================================================================
# 4. FINAL VERDICT
# ==============================================================================
error_margin = 50.0 # Allow some deviation due to fixed-point rounding
diff_2 = abs(sw_res_2 - hw_res_2)

print("\n------------------------------------------------")
if diff_2 < error_margin:
    print("✅ TEST PASSED: Hardware matches Software within tolerance.")
else:
    print("❌ TEST FAILED: Discrepancy is too large.")
    print("   Check if SW_BOOST needs to be higher or lower.")
print("------------------------------------------------")

--- STARTING VERIFICATION ---
1. Loading NPZ Data...
   Note: Slicing eigen_vecs from (10000, 2) to 1D.
2. Initializing Hardware...
   Hardware Loaded.

3. Running Comparison Tests...

[TEST 1] Input = Mean Face
   SW Result: 0.0000
   HW Result: 0.0000
   Diff:      0.0000

[TEST 2] Input = Random Noise (0-255)
   SW Result: -321.5553
   HW Result: -320.0289
   Diff:      1.5264

[TEST 3] Input = Mean + Weights * 1000
   SW Result: 999.9990
   HW Result: 982.6086
   Diff:      17.3904

------------------------------------------------
✅ TEST PASSED: Hardware matches Software within tolerance.
------------------------------------------------


In [6]:
pip show numpy


Name: numpy
Version: 1.21.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/lib/python3/dist-packages
Requires: 
Required-by: deltasigma, jupyterplot, patsy, pynq, pynqutils
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from pynq import Overlay, allocate
import time

# ==============================================================================
# 1. HARDWARE DRIVER 
# ==============================================================================
class FisherfaceDriver:
    def __init__(self, bitfile_path):
        self.overlay = Overlay(bitfile_path)
        self.dma = self.overlay.axi_dma_0
        self.ip = self.overlay.fisherface_accel_0
        
        self.VECTOR_SIZE = 10000
        self.HW_SCALE = 128.0
        self.SW_BOOST = 20.0 
        self.TOTAL_SCALE = self.HW_SCALE * self.SW_BOOST
        
        self.input_buffer = allocate(shape=(self.VECTOR_SIZE,), dtype=np.int32)
        
    def _send_chunked(self, data_array, is_weight=False):
        flat_data = data_array.flatten()
        scale = self.TOTAL_SCALE if is_weight else self.HW_SCALE
        quantized = (flat_data * scale).astype(np.int32)
        np.copyto(self.input_buffer, quantized)
        
        CHUNK_SIZE = 4000
        total_elements = self.VECTOR_SIZE
        for i in range(0, total_elements, CHUNK_SIZE):
            end = min(i + CHUNK_SIZE, total_elements)
            self.dma.sendchannel.transfer(self.input_buffer[i:end])
            self.dma.sendchannel.wait()

    def load_mean(self, mean_vector):
        self.ip.register_map.mode.mode = 1
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(mean_vector, is_weight=False)
        
    def load_weights(self, weight_vector):
        self.ip.register_map.mode.mode = 2
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(weight_vector, is_weight=True)
        
    def inference(self, face_vector):
        self.ip.register_map.mode.mode = 0
        self.ip.register_map.CTRL.AP_START = 1
        self._send_chunked(face_vector, is_weight=False)
        
        while self.ip.register_map.CTRL.AP_DONE == 0: pass
            
        raw_int = int(self.ip.register_map.output_score.output_score)
        if raw_int >= 0x80000000: raw_int -= 0x100000000
        
        acc_scale = 65536.0 # 2^16
        result_float = raw_int / (acc_scale * self.SW_BOOST)
        return result_float

# ==============================================================================
# 2. SOFTWARE IMPLEMENTATION
# ==============================================================================
def software_inference(face, mean, weights):
    centered = face - mean
    result = np.dot(centered, weights)
    return result

# ==============================================================================
# 3. TEST HARNESS
# ==============================================================================

print("--- STARTING DYNAMIC VERIFICATION ---")

# A. LOAD DATA
print("1. Loading NPZ Data...")
data = np.load("face_model_data2.npz", allow_pickle=True)
mean_vec = data['mean_vec']
eigen_vecs = data['eigen_vecs']

if eigen_vecs.ndim > 1:
    w_vec = eigen_vecs[:, 0]
else:
    w_vec = eigen_vecs

# B. INIT HARDWARE
print("2. Initializing Hardware...")
driver = FisherfaceDriver("fisherface.bit")
driver.load_mean(mean_vec)
driver.load_weights(w_vec)
print("   Hardware Loaded.")

# C. RUN TESTS
print("\n3. Running Comparison Tests...")

# --- TEST 1: Mean Face (Consistent) ---
# We verify this once to ensure baseline is zero
print("\n[TEST 1] Input = Mean Face (Baseline)")
hw_res = driver.inference(mean_vec)
print(f"   HW Result: {hw_res:.4f} (Expected ~0.0)")


# --- TEST 2: Random Noise (5 Different Inputs) ---
print("\n[TEST 2] Input = Different Random Noise (0-255)")
error_margin = 50.0
passed_2 = True

for i in range(5):
    # Generate NEW random input every time
    # Note: We don't seed here, or we use a changing seed (i) to ensure variety
    np.random.seed(i) 
    input_random = np.random.rand(10000) * 255.0
    
    sw_res = software_inference(input_random, mean_vec, w_vec)
    hw_res = driver.inference(input_random)
    diff = abs(sw_res - hw_res)
    
    print(f"   [Run {i+1}] SW: {sw_res:9.4f} | HW: {hw_res:9.4f} | Diff: {diff:.4f}")
    if diff > error_margin: passed_2 = False


# --- TEST 3: Synthetic Face (5 Different Multipliers) ---
# We scale the weights by different amounts (100, 500, 1000...) to test range
print("\n[TEST 3] Input = Mean + Weights * Variable_Scale")
passed_3 = True
scales = [100.0, 500.0, 1000.0, -500.0, -1000.0]

for i, scale in enumerate(scales):
    # Create NEW input based on scale
    input_syn = mean_vec + (w_vec * scale)
    
    sw_res = software_inference(input_syn, mean_vec, w_vec)
    hw_res = driver.inference(input_syn)
    diff = abs(sw_res - hw_res)
    
    print(f"   [Run {i+1} (Scale={scale:5})] SW: {sw_res:9.4f} | HW: {hw_res:9.4f} | Diff: {diff:.4f}")
    if diff > error_margin: passed_3 = False

# ==============================================================================
# 4. FINAL VERDICT
# ==============================================================================
print("\n------------------------------------------------")
if passed_2 and passed_3:
    print("✅ TEST PASSED: Hardware matches Software across VARIOUS inputs.")
else:
    print("❌ TEST FAILED: Discrepancy detected.")
print("------------------------------------------------")

--- STARTING DYNAMIC VERIFICATION ---
1. Loading NPZ Data...
2. Initializing Hardware...
   Hardware Loaded.

3. Running Comparison Tests...

[TEST 1] Input = Mean Face (Baseline)
   HW Result: 0.0000 (Expected ~0.0)

[TEST 2] Input = Different Random Noise (0-255)
   [Run 1] SW: -364.9140 | HW: -363.5992 | Diff: 1.3148
   [Run 2] SW: -174.7196 | HW: -174.1816 | Diff: 0.5380
   [Run 3] SW: -402.6075 | HW: -399.6539 | Diff: 2.9536
   [Run 4] SW: -307.2051 | HW: -304.1391 | Diff: 3.0660
   [Run 5] SW: -290.8157 | HW: -286.9887 | Diff: 3.8270

[TEST 3] Input = Mean + Weights * Variable_Scale
   [Run 1 (Scale=100.0)] SW:  100.0003 | HW:   96.7500 | Diff: 3.2503
   [Run 2 (Scale=500.0)] SW:  499.9996 | HW:  490.4250 | Diff: 9.5746
   [Run 3 (Scale=1000.0)] SW:  999.9990 | HW:  982.6086 | Diff: 17.3904
   [Run 4 (Scale=-500.0)] SW: -499.9996 | HW: -494.1020 | Diff: 5.8976
   [Run 5 (Scale=-1000.0)] SW: -999.9991 | HW: -986.2980 | Diff: 13.7011

-----------------------------------------------